# **DataFrame imdb**

## **Import des bibliothèques nécessaires**

In [3]:
import pandas as pd

## **Chargement des datasets IMDb**

In [ ]:
# Nous importons les données directement depuis les fichiers IMDb compressés au format `.tsv.gz`.

#* Datasets principaux
df_title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_title_ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t', compression='gzip', na_values='\\N')

#* Datasets complémentaires
df_title_akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_title_crew = pd.read_csv('https://datasets.imdbws.com/title.crew.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_name_basics = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_title_principals = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', sep='\t', compression='gzip', na_values='\\N')

C:\Users\koke7\AppData\Local\Temp\ipykernel_15176\2730036939.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
C:\Users\koke7\AppData\Local\Temp\ipykernel_15176\2730036939.py:10: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')


## **Nettoyage des datasets IMDb**

In [ ]:

#! === SETUP ===
#? title_basics

# 1. Garder uniquement les films (`titleType = 'movie'`).
# 2. Supprimer les lignes avec des valeurs manquantes essentielles.
# 3. Filtrer les années de production pour garder uniquement celles entre 1970 et 2025.
# 4. Réduire le dataset aux colonnes essentielles.

df_title_basics_clean = (
    df_title_basics[df_title_basics['titleType'] == 'movie']
    .dropna(subset=['startYear', 'genres'])
    .assign(startYear=lambda x: x['startYear'].astype(int))
    .query('1970 <= startYear <= 2025')
    [['tconst', 'primaryTitle']]
)


#! === SETUP ===
#? title_rating

# 1. Garder uniquement les films ayant plus de 1000 votes.

df_title_ratings_clean = df_title_ratings[df_title_ratings['numVotes'] > 1000]


#! === SETUP ===
#? title_akas

# 1. Garder uniquement les titres traduits en français (`region = 'FR'`).
# 2. Réduire aux colonnes essentielles et renommer pour correspondre au format attendu.

df_title_akas_clean = (
    df_title_akas[df_title_akas['region'] == 'FR']
    [['titleId', 'title']]
    .rename(columns={'titleId': 'tconst', 'title': 'Titre Français'})
)


#! === SETUP ===
#? title_crew

# 1. Remplir les valeurs manquantes dans `directors` avec "Unknown".
# 2. Transformer `directors` en une liste.
# 3. Exploser les réalisateurs en lignes individuelles pour fusionner avec `name.basics`.
# 4. Regrouper les réalisateurs par film sous forme de liste.

df_title_crew_clean = (
    df_title_crew.fillna({'directors': 'Unknown'})
    .assign(directors=lambda x: x['directors'].str.split(','))
    .explode('directors')
    .merge(df_name_basics[['nconst', 'primaryName']], left_on='directors', right_on='nconst', how='left')
    .groupby('tconst')['primaryName'].apply(list).reset_index()
)


#! === SETUP ===
#? title_principals

# 1. Filtrer les acteurs uniquement.
# 2. Fusionner avec `name.basics` pour récupérer les noms des acteurs.
# 3. Regrouper les acteurs par film sous forme de liste.

df_title_principals_clean = (
    df_title_principals.dropna(subset=['category'])
    .query("category == 'actor'")
    .merge(df_name_basics[['nconst', 'primaryName']], on='nconst', how='left')
    .groupby('tconst')['primaryName'].apply(list).reset_index()
)

## **Fusion des datasets IMDb**

In [ ]:

#! === MERGEUP ===
#* Nous fusionnons les datasets nettoyés pour créer un seul dataframe regroupant toutes les informations utiles.

#* Fusion `title.basics` et `title.ratings`
df_merged_v1 = df_title_basics_clean.merge(df_title_ratings_clean, on='tconst')

#* Fusion avec `title.akas`
df_merged_v2 = df_merged_v1.merge(df_title_akas_clean, on='tconst', how='left')

#* Fusion avec `title.crew`
df_merged_v3 = df_merged_v2.merge(df_title_crew_clean, on='tconst', how='left')

#* Ajout des acteurs depuis `title.principals`
df_merged_v3['Acteurs'] = df_merged_v3['tconst'].map(dict(zip(df_title_principals_clean['tconst'], df_title_principals_clean['primaryName'])))

#* Export du dataframe IMDb final
df_merged_v3.to_csv("../data/raw/df_movie.csv", index=False)

# **Initialisation de TMDB**

## **Chargement des données**

In [41]:
import pandas as pd
import requests

# Clé API TMDb
api_key = 'f26ef44bcadc5d6ffa22263ea37741ce'

# === IMPORT ===
df_movie = pd.read_csv('../data/raw/df_movie.csv')
df_tmdb = pd.read_csv('../data/raw/tmdb_full.csv')

C:\Users\koke7\AppData\Local\Temp\ipykernel_10176\3087420054.py:9: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmdb = pd.read_csv('../data/raw/tmdb_full.csv')


## **Initialisation des datasets**

In [42]:

#* Nous conservons uniquement les colonnes pertinentes pour le projet.
df_tmdb = df_tmdb[
    ['id', 'backdrop_path', 'budget', 'genres', 'imdb_id', 'original_language', 'overview', 'popularity', 
     'poster_path', 'production_countries', 'release_date', 'revenue', 'runtime', 
     'spoken_languages', 'vote_average', 'vote_count', 'production_companies_name']
]

#* Renommage des colonnes pour le dataset IMDb (df_movie)
df_movie.rename(columns={
    'tconst': 'ID imdb',
    'primaryTitle': 'Titre Original',
    'averageRating': 'Note imdb',
    'numVotes': 'Votes imdb',
    'title': 'Titre Français',
    'actors': 'Acteurs',
    'primaryName': 'Réalisateur(s)'
}, inplace=True)

#* Mise en forme des colonnes "Réalisateur(s)" et "Acteurs" pour convertir les listes en chaînes lisibles.
df_movie['Réalisateur(s)'] = df_movie['Réalisateur(s)'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_movie['Réalisateur(s)'] = df_movie['Réalisateur(s)'].str.strip("[]").str.replace("'", "")

df_movie['Acteurs'] = df_movie['Acteurs'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_movie['Acteurs'] = df_movie['Acteurs'].str.strip("[]").str.replace("'", "")

#* Renommage des colonnes pour le dataset TMDB (df_tmdb)
df_tmdb.rename(columns={
    'id': 'ID tmdb',
    'backdrop_path': 'Image de Fond',
    'budget': 'Budget',
    'genres': 'Genres',
    'overview': 'Synopsis',
    'original_language': 'Langue Originale',
    'popularity': 'Popularité',
    'poster_path': 'Affiche',
    'production_countries': 'Pays de Production',
    'release_date': 'Date de Sortie',
    'revenue': 'Box Office',
    'runtime': 'Durée',
    'spoken_languages': 'Langues Parlées',
    'vote_average': 'Note tmdb',
    'vote_count': 'Votes tmdb',
    'production_companies_name': 'Compagnies de Production'
}, inplace=True)

#* Mise en forme des colonnes avec des listes pour les rendre lisibles.
df_tmdb['Pays de Production'] = df_tmdb['Pays de Production'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Pays de Production'] = df_tmdb['Pays de Production'].str.strip("[]").str.replace("'", "")

df_tmdb['Compagnies de Production'] = df_tmdb['Compagnies de Production'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Compagnies de Production'] = df_tmdb['Compagnies de Production'].str.strip("[]").str.replace("'", "")

df_tmdb['Langues Parlées'] = df_tmdb['Langues Parlées'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Langues Parlées'] = df_tmdb['Langues Parlées'].str.strip("[]").str.replace("'", "")

df_tmdb['Genres'] = df_tmdb['Genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Genres'] = df_tmdb['Genres'].str.strip("[]").str.replace("'", "")

## **Fusion de Imdb et Tmdb**

In [47]:

#! === MERGEUP ===

df_final = pd.merge(df_movie, df_tmdb, left_on='ID imdb', right_on='imdb_id')

## **Filtrage Général**

In [49]:

#* Critères de sélection pour inclure uniquement les films pertinents dans l'analyse.
df_movie_cleaned_v1 = df_final[
    # Films en français avec plus de 5000 votes et popularité > 2.5
    ((df_final['Votes imdb'] > 5000) & (df_final['Popularité'] > 2.5) & (df_final['Langue Originale'] == 'fr')) |
    # Films avec des langues parlées incluant le français, plus de 15000 votes et popularité > 7.5
    ((df_final['Votes imdb'] > 15000) & (df_final['Popularité'] > 7.5) & (df_final['Langues Parlées'].apply(lambda x: 'fr' in x))) |
    # Films en anglais avec plus de 30000 votes et popularité > 15
    ((df_final['Votes imdb'] > 30000) & (df_final['Popularité'] > 15) & (df_final['Langue Originale'] == 'en')) |
    # Films avec au moins 50000 votes et popularité > 25
    (df_final['Votes imdb'] > 50000) & (df_final['Popularité'] > 25)
]


#* On conserve uniquement les films ayant des notes IMDb et TMDb supérieures à 4.
df_movie_cleaned_v2 = df_movie_cleaned_v1[
    (df_movie_cleaned_v1['Note imdb'] > 4) & (df_movie_cleaned_v1['Note tmdb'] > 4)
]


#* Exclusion des films ayant une note IMDb inférieure à 8 et sortis avant l'année 2000.
df_movie_cleaned_v3 = df_movie_cleaned_v2[
    ~((df_movie_cleaned_v2['Note imdb'] < 8) & (df_movie_cleaned_v2['Date de Sortie'] < '2000-01-01'))
]


#* On élimine les doublons en se basant sur le titre original.
df_movie_cleaned_v4 = df_movie_cleaned_v3.drop_duplicates(subset='Titre Original')

#* Création d'une copie explicite pour éviter des avertissements potentiels de pandas.
df_movie_cleaned_v4 = df_movie_cleaned_v4.copy()

#* Conversion de la colonne 'Durée' en format numérique.
df_movie_cleaned_v4['Durée'] = pd.to_numeric(df_movie_cleaned_v4['Durée'], errors='coerce')

## **Ajout de données supplémentaires**

In [51]:

#* Cette colonne catégorise les films en fonction de leur nombre de votes IMDb.
def categorize_votes(votes):
    if votes < 50000:
        return "Connu"
    elif 50000 <= votes < 200000:
        return "Populaire"
    elif 200000 <= votes < 1000000:
        return "Très populaire"
    else:
        return "Blockbuster"

df_movie_cleaned_v4['Réputation'] = df_movie_cleaned_v4['Votes imdb'].apply(categorize_votes)


#* Cette colonne catégorise les films selon leur durée en minutes.
def categorize_times(duree):
    if duree < 100:
        return "Court"
    elif 100 <= duree <= 200:
        return "Moyen"
    else:
        return "Long"

df_movie_cleaned_v4['Métrage'] = df_movie_cleaned_v4['Durée'].apply(categorize_times)


#* Cette colonne regroupe les films par décennie de sortie.
def categorize_years(year):
    if 1970 <= year <= 1979:
        return "70°s"
    if 1980 <= year <= 1989:
        return "80°s"
    if 1990 <= year <= 1999:
        return "90°s"
    if 2000 <= year <= 2009:
        return "2000"
    if 2010 <= year <= 2019:
        return "2010"
    if 2020 <= year <= 2029:
        return "2020"

# Fonction pour récupérer les mots-clés d'un film par ID
def get_movie_keywords(movie_id):
    url_keywords = f'https://api.themoviedb.org/3/movie/{movie_id}/keywords?api_key={api_key}&language=en-US'
    response_keywords = requests.get(url_keywords)
    keywords_data = response_keywords.json()
    keywords = [kw['name'] for kw in keywords_data.get('keywords', [])]
    return keywords

# Ajouter les mots-clés à chaque film dans le DataFrame TMDb complet
df_movie_cleaned_v4['Mots-Clés'] = df_movie_cleaned_v4['ID tmdb'].apply(get_movie_keywords)


df_movie_cleaned_v4['Décennie'] = df_movie_cleaned_v4['Date de Sortie'].apply(
    lambda x: categorize_years(int(str(x)[:4])) if isinstance(x, str) else x
)


#* Cette colonne extrait le premier genre de la liste des genres.
df_movie_cleaned_v4['Genre Principal'] = df_movie_cleaned_v4['Genres'].apply(
    lambda x: x.split(',')[0] if isinstance(x, str) else x
)


#* On exclut les films dont le genre principal est 'Documentary'.
df_movie_cleaned_v4 = df_movie_cleaned_v4[df_movie_cleaned_v4['Genre Principal'] != 'Documentary']


#* On repositionne les colonnes pour une meilleure lisibilité et cohérence.
df_movie_cleaned_v4 = df_movie_cleaned_v4[
    ['ID imdb', 'ID tmdb', 'Titre Original', 'Titre Français', 'Réalisateur(s)', 'Acteurs',
     'Budget', 'Genres', 'Mots-Clés', 'Genre Principal', 'Date de Sortie', 'Décennie',
     'Langue Originale', 'Langues Parlées', 'Synopsis', 'Popularité', 'Réputation',
     'Affiche', 'Image de Fond', 'Durée', 'Métrage',
     'Note tmdb', 'Votes tmdb', 'Note imdb', 'Votes imdb',
     'Compagnies de Production', 'Pays de Production', 'Box Office']
]


## **Exportation**

In [53]:

#! === EXPORT ===

df_movie_cleaned_v4.to_csv("../data/processed/df_movie_cleaned.csv", index=False)

## **Rendu :**

In [2]:
import pandas as pd

df = pd.read_csv("../data/processed/df_movie_cleaned.csv")

In [3]:
df

,ID imdb,ID tmdb,Titre Original,Titre Français,Réalisateur(s),Acteurs,Budget,Genres,Mots-Clés,Genre Principal,...,Image de Fond,Durée,Métrage,Note tmdb,Votes tmdb,Note imdb,Votes imdb,Compagnies de Production,Pays de Production,Box Office
0,tt0035423,11232,Kate & Leopold,Kate et Léopold,James Mangold,"Hugh Jackman, Liev Schreiber, Breckin Meyer, B...",48000000,"Romance, Fantasy, Comedy","['new york city', 'time travel', 'duke', 'fish...",Romance,...,/hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg,118,Moyen,6.326,1187,6.4,90717,"Konrad Pictures, Miramax",US,76019048
1,tt0066921,185,A Clockwork Orange,Orange mécanique,Stanley Kubrick,"Malcolm McDowell, Patrick Magee, Michael Bates...",2200000,"Science Fiction, Drama","['london, england', 'robbery', 'street gang', ...",Science Fiction,...,/3hwKS7e21hzEnXZaOs2FE6e97bc.jpg,137,Moyen,8.218,11572,8.2,898411,"Warner Bros. Pictures, Hawk Films","GB, US",26589000
2,tt0067093,14811,Fiddler on the Roof,Un violon sur le toit,Norman Jewison,"Topol, Leonard Frey, Paul Mann, Paul Michael G...",9000000,"Drama, Romance","['dream', 'parent child relationship', 'tradit...",Drama,...,/rqnOgJWvlURkcOWfGqiACvkTAqG.jpg,179,Moyen,7.700,475,8.0,48843,"United Artists, Cartier Productions, The Miris...",US,83304330
3,tt0068646,238,The Godfather,Le Parrain,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",6000000,"Drama, Crime","['based on novel or book', 'loss of loved one'...",Drama,...,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,175,Moyen,8.712,17871,9.2,2072843,"Paramount, Alfran Productions",US,245066411
4,tt0070047,9552,The Exorcist,L'Exorciste,William Friedkin,"Max von Sydow, Lee J. Cobb, Jack MacGowran, Ja...",15000000,Horror,"['religion and supernatural', 'exorcism', 'hol...",Horror,...,/xcjJ5khg2yzOa282mza39Lbrm7j.jpg,122,Moyen,7.719,6959,8.1,468748,"Hoya Productions, Warner Bros. Pictures",US,441405644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007,tt9806192,586940,I Lost My Body,J'ai perdu mon corps,Jérémy Clapin,"Hakim Faris, ""Patrick dAssumçao"", Alfonso Arfi...",0,"Animation, Drama, Fantasy","['based on novel or book', 'cartoon', 'surreal...",Animation,...,/rHJX9CMYk7FXxUV5la1dEFSPVCv.jpg,81,Court,7.506,1117,7.5,37366,"Studio Xilam, Auvergne-Rhône-Alpes Cinéma",FR,0
3008,tt9844522,585216,Escape Room: Tournament of Champions,Escape Game 2 : Gagner n'était que le début,Adam Robitel,"Logan Miller, Thomas Cocquerel, Carlito Oliver...",15000000,"Horror, Thriller, Mystery","['sequel', 'no escape', 'escape room', 'death ...",Horror,...,/dsdbViTNjLu4DbgkkYmuY4xDQ20.jpg,88,Court,6.700,1531,5.7,62564,"Original Film, Columbia Pictures, Shaken, Not ...",US,65774490
3009,tt9848626,585083,Hotel Transylvania 4: Transformania,Hôtel Transylvanie : Changements monstres,"Derek Drymon, Jennifer Kluska","Andy Samberg, Jim Gaffigan, Steve Buscemi, Dav...",0,"Animation, Comedy, Family, Fantasy, Adventure","['monster', 'vampire', 'transformation', 'afte...",Animation,...,/7QabKu8tizoqy8qCZJXljdSpP4A.jpg,87,Court,7.046,1503,6.0,41822,"Sony Pictures Animation, Sony Pictures, MRC",US,0
3010,tt9866072,615665,Holidate,Holidate,John Whitesell,"Luke Bracey, Andrew Bachelor, Manish Dayal, Al...",0,"Comedy, Romance","['date', 'holiday', 'single', 'family dinner',...",Comedy,...,/8veOfB9RbSzFki0Rq3IQIGsFfhC.jpg,104,Moyen,7.055,1907,6.2,81534,Wonderland Sound and Vision,US,0
